In [1]:
import sys
sys.path.insert(1, './env')
sys.path.insert(1, './agents')

import gym
import pylab
import random
import aa_gun
import strategy_imitation, sarsa
import numpy as np
from collections import deque
import keras
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
import matplotlib.pyplot as plt

from tensorflow.keras.applications import resnet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model

EPISODES = 30000
#pip install git+git://github.com/jaredwinick/img2vec-keras.git

Using TensorFlow backend.


In [ ]:
_IMAGE_NET_TARGET_SIZE = (224, 224)

class Img2Vec(object):
    def __init__(self):
        model = resnet50.ResNet50(weights='imagenet')
        layer_name = 'avg_pool'
        self.intermediate_layer_model = Model(inputs=model.input, 
                                              outputs=model.get_layer(layer_name).output)

    def get_vec(self, image_source, is_file=False,resize_type='crop'):
        """ Gets a vector embedding from an image.
        :param image_path: path to image on filesystem
        :returns: numpy ndarray
        """
        if is_file:
            img = image.load_img(image_source, target_size=_IMAGE_NET_TARGET_SIZE)
            x = image.img_to_array(img)
        else:
            #is array. image.array_to_img(state) to show
            if resize_type=='resize':
                img = image.array_to_img(image_source) 
                img=img.resize(_IMAGE_NET_TARGET_SIZE)
                x = image.img_to_array(img)
            elif resize_type=='crop':
                x=np.zeros([_IMAGE_NET_TARGET_SIZE[0],_IMAGE_NET_TARGET_SIZE[1],3])
                x[:image_source.shape[0],:image_source.shape[1],:]=image_source
        x = np.expand_dims(x, axis=0)
        x = resnet50.preprocess_input(x)
        intermediate_output = self.intermediate_layer_model.predict(x)
        return intermediate_output[0]

In [ ]:
# In case of CartPole-v1, maximum length of episode is 500
#env = aa_gun.AA_gun_simple0_env()
env = gym.make('Seaquest-v0')

#env=CartPoleEnv9()
# get size of state and action from environment
state_size = 2048 #размер эмбеддинга
action_size = env.action_space.n

#agent = DoubleDQNAgent(state_size, action_size)
agent = sarsa.SarsaAgent(state_size, action_size,layers_size=[50,200])
agent.train_start=3000
#agent = strategy_imitation.ImitAgent(state_size, action_size)
agent.render=True

scores, episodes = [], []
reward_lst = []
s_list=[]
a_list=[]

i2v=Img2Vec()

for e in range(EPISODES):
    done = False
    score = 0
    state = env.reset()

    while not done:
        if (e in range(100,300)) or (e in range(300,306)) or (e in range(400,406)) or (e in range(500,506)) or (e in range(600,604)):
            if agent.render:
                env.render()

        # get action for the current state and go one step in environment
        if state.shape!=(1,2048):
            state=i2v.get_vec(state)
        state = np.reshape(state, [1, state_size])
        action = agent.get_action(state)
        next_state, reward, done, info = env.step(action)
        
        next_state=i2v.get_vec(next_state)
        next_state = np.reshape(next_state, [1, state_size])
        # if an action make the episode end, then gives penalty of -100
        

        # save the sample <s, a, r, s'> to the replay memory
        reward_curr=reward
        #if done:
        #    reward_curr -= 100
        agent.append_sample(state, action, reward_curr, next_state, done)
        #if next_state[0,11]!=reward:
        #    print('state[13]!=reward',state[0,11],reward)
        #
        s_list.append(state)
        a_list.append(action)
        reward_lst.append(reward)
        #
        
        # every time step do the training
        agent.train_model()
        score += reward
        state = next_state

        if done:
            # every episode update the target model to be same with model
            agent.update_target_model()

            # every episode, plot the play time
            scores.append(score)
            episodes.append(e)
            pylab.plot(episodes, scores, 'b')
            #pylab.savefig("./save_graph/aa_gun_dqn.png")
            print("episode:", e, "  score:", score,np.mean(scores), "  memory length:",
                  len(agent.memory), "  epsilon:", agent.epsilon)

            # if the mean of scores of last 10 episode is bigger than 490
            # stop training

    # save the model
    #if e % 50 == 0:
    #    agent.model.save_weights("./save_model/aa_gun_dqn.h5")

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 50)                102450    
_________________________________________________________________
batch_normalization_5 (Batch (None, 50)                200       
_________________________________________________________________
dropout_5 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 200)               10200     
_________________________________________________________________
batch_normalization_6 (Batch (None, 200)               800       
_________________________________________________________________
dropout_6 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 1)                

Epoch 2/2
500/500 [==============================] - 0s 28us/step - loss: 457.6176
Epoch 1/2
500/500 [==============================] - 0s 27us/step - loss: 454.2953
Epoch 2/2
500/500 [==============================] - 0s 28us/step - loss: 459.5378
Epoch 1/2
500/500 [==============================] - 0s 27us/step - loss: 454.5728
Epoch 2/2
500/500 [==============================] - 0s 28us/step - loss: 453.3684
Epoch 1/2
500/500 [==============================] - 0s 26us/step - loss: 449.6866
Epoch 2/2
500/500 [==============================] - 0s 27us/step - loss: 445.1490
Epoch 1/2
500/500 [==============================] - 0s 27us/step - loss: 441.9534
Epoch 2/2
500/500 [==============================] - 0s 27us/step - loss: 448.9604
Epoch 1/2
500/500 [==============================] - 0s 56us/step - loss: 442.6811
Epoch 2/2
500/500 [==============================] - 0s 54us/step - loss: 438.5895
Epoch 1/2
500/500 [==============================] - 0s 38us/step - loss: 445.0113
Epoc

Epoch 2/2
500/500 [==============================] - 0s 49us/step - loss: 575.1848
Epoch 1/2
500/500 [==============================] - 0s 45us/step - loss: 565.4849
Epoch 2/2
500/500 [==============================] - 0s 32us/step - loss: 562.6504
Epoch 1/2
500/500 [==============================] - 0s 29us/step - loss: 562.1943
Epoch 2/2
500/500 [==============================] - 0s 30us/step - loss: 563.6550
Epoch 1/2
500/500 [==============================] - 0s 29us/step - loss: 554.2780
Epoch 2/2
500/500 [==============================] - 0s 30us/step - loss: 559.7402
Epoch 1/2
500/500 [==============================] - 0s 28us/step - loss: 549.3281
Epoch 2/2
500/500 [==============================] - 0s 28us/step - loss: 540.6892
Epoch 1/2
500/500 [==============================] - 0s 29us/step - loss: 538.6960
Epoch 2/2
500/500 [==============================] - 0s 30us/step - loss: 536.3162
Epoch 1/2
500/500 [==============================] - 0s 32us/step - loss: 530.5129
Epoc

Epoch 2/2
500/500 [==============================] - 0s 27us/step - loss: 218.3906
Epoch 1/2
500/500 [==============================] - 0s 26us/step - loss: 213.4860
Epoch 2/2
500/500 [==============================] - 0s 28us/step - loss: 209.0783
Epoch 1/2
500/500 [==============================] - 0s 28us/step - loss: 211.1976
Epoch 2/2
500/500 [==============================] - 0s 28us/step - loss: 214.6322
Epoch 1/2
500/500 [==============================] - 0s 47us/step - loss: 209.0441
Epoch 2/2
500/500 [==============================] - 0s 45us/step - loss: 208.8931
Epoch 1/2
500/500 [==============================] - 0s 37us/step - loss: 209.9664
Epoch 2/2
500/500 [==============================] - 0s 35us/step - loss: 204.2869
Epoch 1/2
500/500 [==============================] - 0s 33us/step - loss: 202.2999
Epoch 2/2
500/500 [==============================] - 0s 35us/step - loss: 201.4824
Epoch 1/2
500/500 [==============================] - 0s 34us/step - loss: 201.3218
Epoc

Epoch 2/2
500/500 [==============================] - 0s 33us/step - loss: 68.2622
delta_r 7.249481299128236 r 12.999234252582973
Epoch 1/2
500/500 [==============================] - 0s 31us/step - loss: 314.8094
Epoch 2/2
500/500 [==============================] - 0s 28us/step - loss: 310.3871
Epoch 1/2
500/500 [==============================] - 0s 27us/step - loss: 314.6475
Epoch 2/2
500/500 [==============================] - 0s 29us/step - loss: 306.0013
Epoch 1/2
500/500 [==============================] - 0s 29us/step - loss: 310.1949
Epoch 2/2
500/500 [==============================] - 0s 29us/step - loss: 306.4259
Epoch 1/2
500/500 [==============================] - 0s 27us/step - loss: 299.2556
Epoch 2/2
500/500 [==============================] - 0s 27us/step - loss: 295.1009
Epoch 1/2
500/500 [==============================] - 0s 29us/step - loss: 291.5463
Epoch 2/2
500/500 [==============================] - 0s 29us/step - loss: 282.9217
Epoch 1/2
500/500 [======================

...

delta_r 6.017732145678011 r 13.536387273684419
Epoch 1/2
500/500 [==============================] - 0s 28us/step - loss: 33.7901
Epoch 2/2
500/500 [==============================] - 0s 28us/step - loss: 31.9324
Epoch 1/2
500/500 [==============================] - 0s 27us/step - loss: 34.9811
Epoch 2/2
500/500 [==============================] - 0s 28us/step - loss: 31.4773
Epoch 1/2
500/500 [==============================] - 0s 27us/step - loss: 31.7861
Epoch 2/2
500/500 [==============================] - 0s 31us/step - loss: 32.6866
Epoch 1/2
500/500 [==============================] - 0s 28us/step - loss: 30.3088
Epoch 2/2
500/500 [==============================] - 0s 31us/step - loss: 31.6780
Epoch 1/2
500/500 [==============================] - 0s 26us/step - loss: 30.6508
Epoch 2/2
500/500 [==============================] - 0s 28us/step - loss: 30.8111
Epoch 1/2
500/500 [==============================] - 0s 27us/step - loss: 30.6823
Epoch 2/2
500/500 [==============================] 

Epoch 2/2
500/500 [==============================] - 0s 29us/step - loss: 25.0693
Epoch 1/2
500/500 [==============================] - 0s 28us/step - loss: 18.0959
Epoch 2/2
500/500 [==============================] - 0s 35us/step - loss: 16.8069
Epoch 1/2
500/500 [==============================] - 0s 27us/step - loss: 17.0891
Epoch 2/2
500/500 [==============================] - 0s 28us/step - loss: 17.4459
Epoch 1/2
500/500 [==============================] - 0s 28us/step - loss: 16.1873
Epoch 2/2
500/500 [==============================] - 0s 27us/step - loss: 15.5977
Epoch 1/2
500/500 [==============================] - 0s 26us/step - loss: 16.0501
Epoch 2/2
500/500 [==============================] - 0s 27us/step - loss: 16.0063
Epoch 1/2
500/500 [==============================] - 0s 26us/step - loss: 15.5917
Epoch 2/2
500/500 [==============================] - 0s 28us/step - loss: 15.3084
Epoch 1/2
500/500 [==============================] - 0s 26us/step - loss: 15.4078
Epoch 2/2
500/50

Epoch 1/2
500/500 [==============================] - 0s 28us/step - loss: 31.1066
Epoch 2/2
500/500 [==============================] - 0s 28us/step - loss: 31.0518
delta_r 4.139750046492406 r 14.55610019908479
Epoch 1/2
500/500 [==============================] - 0s 65us/step - loss: 33.4477
Epoch 2/2
500/500 [==============================] - 0s 56us/step - loss: 29.8227
Epoch 1/2
500/500 [==============================] - 0s 29us/step - loss: 34.2308
Epoch 2/2
500/500 [==============================] - 0s 29us/step - loss: 30.7606
Epoch 1/2
500/500 [==============================] - 0s 28us/step - loss: 33.1759
Epoch 2/2
500/500 [==============================] - 0s 29us/step - loss: 28.6157
Epoch 1/2
500/500 [==============================] - 0s 30us/step - loss: 32.7302
Epoch 2/2
500/500 [==============================] - 0s 30us/step - loss: 30.2767
Epoch 1/2
500/500 [==============================] - 0s 28us/step - loss: 30.1445
Epoch 2/2
500/500 [==============================] -

Epoch 2/2
500/500 [==============================] - 0s 64us/step - loss: 20.1467
Epoch 1/2
500/500 [==============================] - 0s 50us/step - loss: 23.7083
Epoch 2/2
500/500 [==============================] - 0s 31us/step - loss: 22.7170
Epoch 1/2
500/500 [==============================] - 0s 27us/step - loss: 22.7987
Epoch 2/2
500/500 [==============================] - 0s 29us/step - loss: 22.9035
Epoch 1/2
500/500 [==============================] - 0s 28us/step - loss: 21.6981
Epoch 2/2
500/500 [==============================] - 0s 31us/step - loss: 20.2880
Epoch 1/2
500/500 [==============================] - 0s 29us/step - loss: 19.7503
Epoch 2/2
500/500 [==============================] - 0s 29us/step - loss: 19.4319
Epoch 1/2
500/500 [==============================] - 0s 28us/step - loss: 18.2244
Epoch 2/2
500/500 [==============================] - 0s 29us/step - loss: 17.3775
Epoch 1/2
500/500 [==============================] - 0s 29us/step - loss: 17.3633
Epoch 2/2
500/50

Epoch 1/2
500/500 [==============================] - 0s 34us/step - loss: 55.8467
Epoch 2/2
500/500 [==============================] - 0s 36us/step - loss: 56.1993
Epoch 1/2
500/500 [==============================] - 0s 34us/step - loss: 55.3289
Epoch 2/2
500/500 [==============================] - 0s 33us/step - loss: 56.0652
delta_r 7.054844381243165 r 14.458154565754745
Epoch 1/1
3475/3475 [==============================] - 0s 27us/step - loss: 25.8710
Epoch 1/1
3475/3475 [==============================] - 0s 18us/step - loss: 15.9900
delta_r 5.816017038743082 r 13.112001112487638
episode: 3   score: 200.0 125.0   memory length: 3475   epsilon: 0.030908048516573555
Epoch 1/1
4327/4327 [==============================] - 0s 31us/step - loss: 30.1312
Epoch 1/1
4327/4327 [==============================] - 0s 24us/step - loss: 21.9284
delta_r 6.008897389835576 r 12.735387422224038
episode: 4   score: 140.0 128.0   memory length: 4327   epsilon: 0.0131785477295791
Epoch 1/1
5078/5078 [====

In [ ]:
def make_sar_table(s,a,r):
    #print(np.array(a,ndmin=2).T.shape)
    #print(np.array(r,ndmin=2).T.shape)
    #print(np.array(s,ndmin=2)[:,0,:].shape)
    return np.hstack( (np.array(s,ndmin=2)[:,0,:],np.array(a,ndmin=2).T,np.array(r,ndmin=2).T) )

In [ ]:
import matplotlib.pyplot as plt
print(np.mean(reward_lst))
plt.plot(reward_lst)
plt.show()
plt.plot(scores)

In [ ]:
plt.plot(agent.r_disco)
#plt.plot(agent.d*20)

In [ ]:
np.mean(reward_lst)

In [ ]:
np.mean(score)

In [ ]:
#Чтобы анализировать разрешимость задачи
def replicate_reward(sar,border=0,wanted_part=0.5):
    part = np.mean(sar[:,-1:]>border)
    if part==0:
        print('ERROR')
        return(sar)
    else:
        while part<wanted_part:
            sar=np.vstack((sar,sar[np.where(sar[:,-1:]>border)[0],:]))
            part = np.mean(sar[:,-1:]>border)
        print(part)
        return(sar)

In [ ]:
sar=make_sar_table(s_list,a_list,reward_lst)
sar=replicate_reward(sar)
X=sar[1:,:]
Y=sar[:-1,:]
Y=sar[:-1,-1:]
Y=sar[1:,-1:]
sar_width=X.shape[1]
nn = Sequential()
nn.add(Dense(200, input_dim=sar_width, activation='relu',
                kernel_initializer='he_uniform',kernel_regularizer=keras.regularizers.l2(0.01)))
nn.add(BatchNormalization())
nn.add(Dropout(0.2))
nn.add(Dense(200, activation='relu',
                kernel_initializer='he_uniform',kernel_regularizer=keras.regularizers.l2(0.01)))
nn.add(Dropout(0.2))
nn.add(BatchNormalization())
nn.add(Dense(200, activation='relu',
                kernel_initializer='he_uniform',kernel_regularizer=keras.regularizers.l2(0.01)))
nn.add(Dropout(0.2))
nn.add(BatchNormalization())
#nn.add(Dense(sar_width, activation='linear',
#                kernel_initializer='he_uniform'))
nn.add(Dense(1, activation='linear',
                kernel_initializer='he_uniform'))

nn.summary()
nn.compile(loss='mse', optimizer=Adam(lr=0.001))
l=X.shape[0]
X_train=X[:int(l/2),:]
Y_train=Y[:int(l/2),:]
X_test=X[int(l/2):,:]
Y_test=Y[int(l/2):,:]
nn.fit(X_train, Y_train, batch_size=1200,epochs=30000, verbose=1)

In [ ]:
Y_pred=np.array(nn.predict(X_test),ndmin=2)
mse = np.mean((Y_pred-Y_test)**2)
print(mse)
rmae=np.mean(np.abs(Y_pred-Y_test))/np.mean(np.abs(Y_test))
print(rmae)
rmae_diversed=np.mean(np.abs(Y_pred-Y_test),axis=0)/np.mean(np.abs(Y_test),axis=0)
print(rmae_diversed)

In [ ]:
colnumn_num=-1
plt.plot(Y_pred[:3000,colnumn_num])
plt.plot(Y_test[:3000,colnumn_num])
plt.show()

In [ ]:
#train (переобучение?)
Y_pred=np.array(nn.predict(X_train),ndmin=2)
mse = np.mean((Y_pred-Y_train)**2)
print(mse)
rmae=np.mean(np.abs(Y_pred-Y_train))/np.mean(np.abs(Y_train))
print(rmae)
rmae_diversed=np.mean(np.abs(Y_pred-Y_train),axis=0)/np.mean(np.abs(Y_train),axis=0)
print(rmae_diversed)

colnumn_num=-1
plt.plot(Y_pred[:1300,colnumn_num])
plt.plot(Y_train[:1300,colnumn_num])
plt.show()

In [ ]:
import xgboost as xgb
xgbparams = {
    'booster':'gbtree',
    'metric':'mse',
    'objective':'reg:squarederror',
    'verbosity':0,
    'max_depth': 7,
    'n_estimators': 90,
    'eta': 0.3,
    'nthreads': 2,
    'seed':0
}
nn=xgb.XGBRegressor(**xgbparams)
nn.fit(X_train[:int(l/4),:], Y_train[:int(l/4),:],
           eval_set=[(X_train[int(l/4):,:], Y_train[int(l/4):,:])],
           verbose=True)

In [ ]:
%%time
for i in range(10):
    i2v.get_vec(next_state,resize_type='crop')